# scipy.optimize

### Adrian Price-Whelan

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy import optimize

This subpackage contains optimization schemes for both scalar and multivariate functions.

--- 

## Function minimization

A number of the routines in this subpackage are for minimizing functions. Let's start with a simple example: a quadratic function.

In [ ]:
def f(x):
    return 3*(x - 0.71)**2

x = np.linspace(-1, 2, 100)
plt.plot(x, f(x), marker=None)

In [ ]:
res = optimize.fmin(f, x0=0., disp=False)
print(res)

---

## Bounded minimization

Let's imagine the function we're trying to minimize has multiple minima, and we'd like to find the minimum between a particular set of bounds. For example, the function below:

In [ ]:
def f2(x):
    return x**4 + (x - 0.5)**3 + 0.7*(x-1)**2

x = np.linspace(-3, 3, 100)
plt.semilogy(x, f2(x), marker=None)

Let's say we want to find the minimum between -2 and -0.5 -- we can use `brent()` to specify a bounds to minimize over:

In [ ]:
res = optimize.brent(f2, brack=(-2,0))
print(res)

---

## Solving functions

`fsolve()` finds the roots of a function $f(x) = 0$ given an initial guess. For example, let's consider the Kepler problem in its simplest form:

$$
M = E - \epsilon \sin E
$$

($M$ is the mean anomaly, $E$ is the eccentric anomaly, and $\epsilon$ is the eccentricity of the orbit). For a given eccentricity and mean anomaly, we can solve for the eccentric anomaly by solving for the roots of:

$$
f(E) = E - \epsilon \sin E - M = 0
$$

In [ ]:
def kepler(E, eps, M):
    return E - eps*np.sin(E) - M

In [ ]:
optimize.fsolve(kepler, x0=0., args=(0.1, 1.))

---

## Least-squares fitting

Minimize the sum of squares of the residuals between function values and some data. This uses the Levenberg-Marquardt algorithm, which works even for nonlinear least-squares problems. If your problem is linear, you should probably just use [linear algebra](http://arxiv.org/abs/1008.4686).

To demonstrate how to use `leastsq()`, let's first generate some data:

In [ ]:
def model(x, A, ph):
    # A : amplitude
    # ph : phase
    return A * np.cos(2*np.pi * x + ph)

In [ ]:
npts = 25
true_A = 1.3
true_ph = 0.35

x = np.random.uniform(0, 3., npts)
y = model(x, true_A, true_ph)

# "observe" the data with some uncertainties
sigma = 0.1
y = np.random.normal(y, sigma)

plt.errorbar(x, y, sigma, linestyle='none', marker='o')

We now define a function to compute the residuals between the data and the model with a given set of parameters (the error function):

In [ ]:
def error_func(params, x, y, sigma):
    return -((y - model(x, *params)) / sigma)**2

In [ ]:
p_opt, ier = optimize.leastsq(error_func, [1., 0.5], args=(x, y, sigma))

In [ ]:
print(p_opt)

In [ ]:
x_opt = np.linspace(x.min(), x.max(), 1000)
y_opt = model(x_opt, *p_opt)
plt.errorbar(x, y, sigma, linestyle='none', marker='o')
plt.plot(x_opt, y_opt, linestyle='-', marker=None)

---

<h1 style='background-color: #cccccc; padding: 15px;'>Exercises</h1>

For the exercises below, we're going to `scipy.optimize.leastsq` to fit an absorption line model to a small section of a spectrum. Let's start by downloading a spectrum to use:

In [ ]:
# both of these are standard-library
import urllib2
from cStringIO import StringIO

# yep, using astropy before we've covered astropy...
from astropy.io import fits

In [ ]:
url = "http://mirror.sdss3.org/sas/dr12/sdss/spectro/redux/26/spectra/2393/spec-2393-54156-0622.fits"
response = urllib2.urlopen(url)
file_obj = StringIO()
file_obj.write(response.read())
file_obj.seek(0)

Now, `file_obj` is a file object that contains the FITS file data from the URL we specified -- we can read the FITS file using Astropy

In [ ]:
hdulist = fits.open(file_obj) # read file object as a FITS file
data = hdulist['coadd'].data # get data from the 'coadd' key of the HDUList

Here we extract the relevant columns out of the HDU containing the spectrum itself:

In [ ]:
wvln = 10**data['loglam'] # log-wavelength (Angstroms)
flux = data['flux'] # flux
ivar = data['ivar'] # inverse variance

plt.figure(figsize=(8,6))
plt.plot(wvln, flux, drawstyle='steps', marker=None)
plt.xlim(3500, 9500)

Select a subset of the data $\pm$100 $\mathring {\rm A}$ around H$\alpha$ (6563 $\mathring {\rm A}$), store these as new variables (e.g., `wvln_Ha` and etc.). Plot the subset:

In [ ]:
halpha = 6563.
# <fill in here>

Re-center the wavelength array so that H$\alpha$ is at 0

We are going to fit the absorption line using a Voigt profile plus a line (for the continuum). Start by writing a function `model()` that, given a set of parameters and a wavelength array, computes our model for the absorption line. I've provided a function below to compute the Voigt profile because it is a bit ugly...

In [ ]:
from scipy.special import wofz

def voigt(wvln, amp, pos, fwhm, shape):
    shape_fac = 1j * shape * np.sqrt(np.log(2.))
    tmp = 1 / wofz(shape_fac).real
    tmp = tmp * amp * wofz(2*np.sqrt(np.log(2.)) * (wvln-pos)/fwhm + shape_fac).real
    return tmp

def model(params, wvln):
    """ 
    The parameters, params, will be:
        
        slope, intercept (for continuum)
        amplitude, position, fwhm, and shape (Voigt profile)
    """
    m,b,amp,pos,fwhm,shape = params
            
    # <fill in here>

Play with the parameters until you get something that qualitatively looks like the line above - do this by over-plotting the model over the 0-centered data:

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(wvln_Ha, flux_Ha, drawstyle='steps-mid', 
         marker=None, linewidth=1., color='#666666')
plt.errorbar(wvln_Ha, flux_Ha, 1/np.sqrt(ivar_Ha), 
             marker='.', linestyle='none', color='k')

# <plot model of the line here>

plt.xlim(wvln_Ha.min(), wvln_Ha.max())
plt.xlabel(r"Wavelength [$\AA$]")
plt.ylabel(r"Flux")

Now we're going to write a function that evaluates the logarithm of the likelihood of the data given a set of model parameters. In this case, we're going to assume we have Gaussian noise, so this is equivalent to chi-squared.

In [ ]:
def ln_likelihood(params, wvln, flux, ivar):
    # <Fill in here>
    pass

Use `optimize.leastsq` to find the best-fit model

In [ ]:
# p_opt,ier = optimize.leastsq(<fill in here>) 

Plot your best-fit model over the data: